In [1]:
from scipy.special import expit
import numpy as np
import tensorly as tl
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from utils_binaryTensor_Sarah import TensorModelWithGenetics, generate_survival_data,\
    survival_likelihood
from tensorly.decomposition import parafac
from tensor_optimization import initialize_tensor_model,optimize_tensor_model,compute_loss,generate_polynomial_bases,create_theta
from scipy.special import legendre
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from tensorClass import TensorOptimizer

In [2]:

# set up the parameter 
# generate the outcomes with Sarah's model
## N is the dimension of subjects
## P is the dimension of X
## D is the dimension of disease
## K is the dimension of topics (or factors)
## R1 and R2 are the dimensions of basis functions
# Set random seed for reproducibility
np.random.seed(42)
N, P, D, K, T = 1000, 20,10,5,10
R1 = R2 = 3
np.random.seed(2)

# generate time points 
T_vec = np.linspace(0, 1, T)

h_1 = 4 * (1 - T_vec)**3  # early peaking
h_2 = 27 * T_vec * (1 - T_vec)**2  # middle peaking
h_3 = 4 * T_vec**3  # late peaking
Phi_T = np.vstack((h_1, h_2, h_3)).T
Phi_T, _ = np.linalg.qr(Phi_T)


# Generate 10 polynomial bases
T = len(T_vec)  # Assuming T_vec is already defined
Phi_T = generate_polynomial_bases(T, 3)


true_model = TensorModelWithGenetics(N, P, D, K, R1, R2, T)
theta_true = true_model.compute_theta()
X = true_model.G # the genetic covariates for each individual

# Generate survival data
pi_true = expit(theta_true)
Y, event_times = generate_survival_data(pi_true, N, D, T,censoring_prob=0.0)


In [3]:
# Flatten event_times to get all event times across all diseases
all_event_times = event_times.flatten()

# Calculate basic statistics
mean_event_time = np.mean(all_event_times)
median_event_time = np.median(all_event_times)
max_event_time = np.max(all_event_times)

# Print summary statistics
print("Summary of Event Times (All Diseases Combined):")
print(f"  Mean: {mean_event_time:.2f}")
print(f"  Median: {median_event_time:.2f}")
print(f"  Max: {max_event_time}")
print()

Summary of Event Times (All Diseases Combined):
  Mean: 1.50
  Median: 0.00
  Max: 10



In [ ]:
# if doing class based
K = 5  # Number of components
type_outcome = 'Survival_Sarah'  # Or 'Gaussian' or 'Binary'
max_iter = 1000
tol = 1e-6

# Initialize the TensorOptimizer
optimizer = TensorOptimizer(Y, event_times, X, K, type_outcome, max_iter=max_iter, tol=tol)

# Run the optimization
B1, B2, final_loss = optimizer.optimize()

# Get the final theta
theta_fit, A1_T, A2_T = optimizer.create_theta()

In [4]:

# Initialize tensor model
B1, B2, theta_CP = initialize_tensor_model(Y, K, X, Phi_T)

# Optimize survival tensor
B1_optimized, B2_optimized, final_loss = optimize_tensor_model(
    Y, event_times, X, B1, B2, Phi_T, Phi_T, type_outcome='Survival_Sarah'
)

0th iteration with loss: 15.628
100th iteration with loss: 9.789
200th iteration with loss: 9.728
300th iteration with loss: 9.687
400th iteration with loss: 9.656
Stop: not enough improvement


In [ ]:
def plot_polynomial_bases(Phi, T_vec):
    plt.figure(figsize=(12, 8))
    for i in range(Phi.shape[1]):
        plt.plot(T_vec, Phi[:, i], label=f'Basis {i+1}')
    plt.title('Polynomial Bases')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# Generate and plot the bases
T = 100  # or whatever your T value is
T_vec = np.linspace(0, 1, T)
Phi_T_estimation = generate_polynomial_bases(T,3 )

T=10

plot_polynomial_bases(Phi_T_estimation, T_vec)

In [ ]:
theta_fit=create_theta(Phi_T,Phi_T,B1_optimized,B2_optimized)

pi_true = expit(theta_true)
pi_fit = expit(theta_fit)

# Plot true vs estimated pi for a few randomly selected individuals and diseases
num_samples = 5
sample_individuals = np.random.choice(N, num_samples, replace=False)
sample_diseases = np.random.choice(D, num_samples, replace=False)

plt.figure(figsize=(15, 15))
for i, (n, d) in enumerate(zip(sample_individuals, sample_diseases)):
    plt.subplot(num_samples, 2, 2*i + 1)
    plt.plot(pi_true[n, d, :], label='True')
    plt.plot(pi_fit[n, d, :], label='Estimated (new)')
    plt.title(f'{n}-th row, {d}-th column', fontsize = 20)
    plt.xlabel('Time')
    plt.legend(prop={'size': 20})
    
    plt.subplot(num_samples, 2, 2*i + 2)
    plt.plot(Y[n, d, :], label='Outcome')
    # plt.plot(pi_true[n, d, :], label='True')
    # plt.plot(pi_CP[n, d, :], label='Estimated (old)')
    plt.title(f'{n}-th row, {d}-th column', fontsize = 20)
    plt.xlabel('Time')
    plt.legend(prop={'size': 20})
plt.tight_layout()
# plt.show()


# plt.savefig("example_TC1.pdf")


In [ ]:

A2_T = tl.tenalg.mode_dot(B2_optimized, Phi_T, 2)  
A1_T = tl.tenalg.mode_dot(B1_optimized, Phi_T, 2) 
# Select 3 random individuals
sample_individuals = np.random.choice(N, 3, replace=False)

# Create a figure
plt.figure(figsize=(12, 8))



# Create a subplot for each individual
for i, individual in enumerate(sample_individuals):
    plt.subplot(3, 1, i+1)
    
    # Create a matrix of event times for this individual
    event_matrix = np.zeros((D, T))
    for d in range(D):
        event_time = event_times[individual, d]
        if event_time < T:  # Check if the event time is within bounds
            event_matrix[d, event_time] = 1
        else:
        # If event_time is T (indicating no event), you might want to handle it differently
        # For example, you could mark the last time point:
            event_matrix[d, T-1] = 0 # or some other value to indicate censoring
    
    # Plot the heatmap
    sns.heatmap(event_matrix, cmap='YlOrRd', cbar=False)
    
    plt.title(f'Individual {individual}')
    plt.xlabel('Time')
    plt.ylabel('Disease')
    plt.yticks(range(D), range(1, D+1))

plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(15, 15))
for i, n in enumerate(sample_individuals):
    plt.subplot(num_samples, 2, 2*i + 1)
    plt.plot(A1_T[n, :, :].T)
    plt.title(f'{n}-th row of U_1: topic-specific loadings (estimated)')
    plt.xlabel('Time')
    plt.legend()
    
    plt.subplot(num_samples, 2, 2*i + 2)
    plt.plot(true_model.lambda_k[n, :, :].T)
    plt.title(f'{n}-th row of U_1: topic-specific loadings (truth)')

plt.tight_layout()
plt.show()

here we show the results of using the stuff from R simulations

In [ ]:
import numpy as np
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri

# Activate automatic conversion between R and NumPy arrays
numpy2ri.activate()

# Load data saved as .rds files
Y = robjects.r['readRDS']('Y.rds')
E_python = robjects.r['readRDS']('E_python.rds')
U2 = robjects.r['readRDS']('U2.rds')
U3 = robjects.r['readRDS']('U3.rds')
Theta = robjects.r['readRDS']('Theta.rds')
X = robjects.r['readRDS']('X.rds')



# Convert to NumPy arrays if needed

Y = np.array(Y)
event_times = np.array(E_python)
event_times = event_times.astype(int)
U2 = np.array(U2)
U3 = np.array(U3)
Theta = np.array(Theta)
pi_true=expit(Theta)
X = np.array(X)
K=3

In [ ]:

# Initialize tensor model
B1, B2, theta_CP = initialize_tensor_model(Y, K, X, Phi_T)

In [ ]:

# Optimize survival tensor
B1_optimized, B2_optimized, final_loss = optimize_tensor_model(
    Y, event_times, X, B1, B2, Phi_T, Phi_T, type_outcome='Survival_Sarah'
)

In [ ]:
Phi_A1 = Phi_A2 = Phi_T
A2_T = tl.tenalg.mode_dot(B2_optimized, Phi_A2, 2)  
A1_T = tl.tenalg.mode_dot(B1_optimized, Phi_A1, 2)  
theta_fit = np.einsum("irt, jrt -> ijt", A1_T, A2_T)

pi_true = expit(Theta)
pi_fit = expit(theta_fit)
N=Y.shape[0]
D=Y.shape[1]
T=Y.shape[2]


In [ ]:
event_times.shape
T

In [ ]:

# Plot true vs estimated pi for a few randomly selected individuals and diseases
num_samples = 5
sample_individuals = np.random.choice(N, num_samples, replace=False)
sample_diseases = np.random.choice(D, num_samples, replace=False)

plt.figure(figsize=(15, 15))
for i, (n, d) in enumerate(zip(sample_individuals, sample_diseases)):
    plt.subplot(num_samples, 2, 2*i + 1)
    plt.plot(pi_true[n, d, :], label='True')
    plt.plot(pi_fit[n, d, :], label='Estimated (new)')
    plt.title(f'{n}-th row, {d}-th column', fontsize = 20)
    plt.xlabel('Time')
    plt.legend(prop={'size': 20})
    
    plt.subplot(num_samples, 2, 2*i + 2)
    plt.plot(Y[n, d, :], label='Outcome')
    # plt.plot(pi_true[n, d, :], label='True')
    # plt.plot(pi_CP[n, d, :], label='Estimated (old)')
    plt.title(f'{n}-th row, {d}-th column', fontsize = 20)
    plt.xlabel('Time')
    plt.legend(prop={'size': 20})
plt.tight_layout()
# plt.show()
# plt.savefig("example_TC1.pdf")


In [ ]:
# Select 3 random individuals
sample_individuals = np.random.choice(N, 3, replace=False)

# Create a figure
plt.figure(figsize=(12, 8))



# Create a subplot for each individual
for i, individual in enumerate(sample_individuals):
    plt.subplot(3, 1, i+1)
    
    # Create a matrix of event times for this individual
    event_matrix = np.zeros((D, T))
    for d in range(D):
        event_time = event_times[individual, d]
        if event_time < T:  # Check if the event time is within bounds
            event_matrix[d, event_time] = 1
        else:
        # If event_time is T (indicating no event), you might want to handle it differently
        # For example, you could mark the last time point:
            event_matrix[d, T-1] = 0 # or some other value to indicate censoring
    
    # Plot the heatmap
    sns.heatmap(event_matrix, cmap='YlOrRd', cbar=False)
    
    plt.title(f'Individual {individual}')
    plt.xlabel('Time')
    plt.ylabel('Disease')
    plt.yticks(range(D), range(1, D+1))

plt.tight_layout()
plt.show()
